In [1]:
%pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install simplejson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pybase-db

Note: you may need to restart the kernel to use updated packages.


In [10]:
# 2.1 - 2.4, Importing Libraries:
import boto3
import base64
import json
import os

# 2.5, Set home:
# If not in a project, set locally:
# HOME = "/Users/nataliahelms/git/genai_research/"
# if in a project, use os (recommended):
HOME = os.environ['HOME'] + '/genai_research/'

# 2.6, Instantiate the Bedrock Runtime Client with a region_name of us-east-1:
runtime = boto3.client("bedrock-runtime",region_name="us-east-1")
MODEL_NAME = "claude-3-opus-20240229"

HOME

'/home/sagemaker-user/genai_research/'

In [13]:
##### For Report App purposes, if we are able to access images of reports via URL, 
##### this functionality would be useful using the requests library:
#image_url = "https://url-to-the-image.com/file.jpg"
# Funtion to encode the image using URL
def encode_image_url(url):
    with requests.get(url).content as image_bytes:
        return(base64.b64encode(image_bytes).decode("utf-8"))

In [14]:
##### However, for our immediate use case, we are using local files so we will use this method:
# Function to encode the image via path
def encode_image_path(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [15]:
# Getting the base64 string
# Image Path
superstore_path = HOME + 'data/images/superstore.png'
ny_housing_path = HOME + 'data/images/housing_new_york.png'
silicon_valley_path = HOME + 'data/images/unaffordable_housing_silicon_valley.png'

# Getting the base64 string
superstore_base64 = encode_image_path(superstore_path)
ny_housing_base64 = encode_image_path(ny_housing_path)
silicon_valley_base64 = encode_image_path(silicon_valley_path)

In [22]:
##### Step 4: Build Inference Parameter Input
####### For Claude 3 Sonnet, we are required to use the Anthropic Claude Messages API as the inference parameter input.
####### Put this in the body parameter using json.dumps
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

# Step 4.1, set the body using paremeter input:
# json.dumps a dictionary of Claude’s required inference parameters
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text", 
                        "text": "I will upload 3 images of Tableau Workbooks. Which ones would fit into the Housing, Geographic, or Affordabilty categories? Images can belong to several or none of the categories, make sure to look."
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": silicon_valley_base64,
                        },
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": ny_housing_base64,
                        },
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": superstore_base64,
                        },
                    },
                ],
            }
        ],
    }
)

In [20]:

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 5: Invoke the Model
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

# Step 5, Invoke the model
response = runtime.invoke_model(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    contentType="application/json",
    accept="application/json",
    body=body
)


In [21]:

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 6: Get the Response
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

# Step 6,
# Convert the StreamingBody to a JSON encoded string using .read() 
# then turn the JSON string into a dictionary using json.loads
response_body = json.loads(response.get("body").read())


#The Claude JSON output includes a completion attribute with the text.
print(response_body.get("content"))


[{'type': 'text', 'text': 'Based on the images provided, here is how they fit into the Housing, Geographic, and Affordability categories:\n\nImage 1:\nThis image belongs to the Housing and Geographic categories. It displays a map of the Silicon Valley region, showing the change in median monthly housing costs from 2010 to 2017 across different census tracts. The geographic map visualization and focus on housing costs relate to both the Geographic and Housing categories.\n\nImage 2:  \nThis image falls under the Housing and Affordability categories. It presents information about New York City\'s plan to expand affordable housing opportunities, including details on the number of affordable housing units targeted and completion rates. The explicit mention of "affordable housing" and the statistics provided align with both the Housing and Affordability categories.\n\nImage 3:\nThis image does not seem to fit into any of the Housing, Geographic, or Affordability categories. It appears to be

Test creating a description prompt:

In [26]:
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 4: Build Inference Parameter Input
####### For Claude 3 Sonnet, we are required to use the Anthropic Claude Messages API as the inference parameter input.
####### Put this in the body parameter using json.dumps
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

# Step 4.1, set the body using paremeter input:
# json.dumps a dictionary of Claude’s required inference parameters
body2 = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text", 
                        "text": "I will upload 1 images of a Tableau Workbook. Can you provide a brief description of what this image of a report contains? Please limit the description to maximum 5 sentences."
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": silicon_valley_base64,
                        },
                    },
                ],
            }
        ],
    }
)
body_keyword = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text", 
                        "text": "I will upload 1 images of a Tableau Workbook. Can you provide a list of 10 key words to describe what type of information is contained in this report? Please limit the description to maximum of 10 keywords."
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": silicon_valley_base64,
                        },
                    },
                ],
            }
        ],
    }
)


In [24]:

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
##### Step 5: Invoke the Model
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 

# Step 5, Invoke the model
response = runtime.invoke_model(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    contentType="application/json",
    accept="application/json",
    body=body2
)


In [25]:
# Step 6,
# Convert the StreamingBody to a JSON encoded string using .read() 
# then turn the JSON string into a dictionary using json.loads
response_body = json.loads(response.get("body").read())


#The Claude JSON output includes a completion attribute with the text.
print(response_body.get("content"))

[{'type': 'text', 'text': 'This image contains a map of the Silicon Valley area, specifically focusing on the change in median monthly housing costs from 2010 to 2017. The map is color-coded, with census tracts shaded in orange indicating an increase in housing costs, and tracts shaded in blue indicating a decrease or lower housing costs.\n\nThe title of the report is "Affordable Housing in Silicon Valley." The map area covers a significant portion of the San Francisco Bay Area, including cities such as San Francisco, Oakland, and San Jose.\n\nAccording to the description provided, approximately 83% of the census tracts shown on the map experienced an increase in monthly housing costs between 2010 and 2017. The Uptown neighborhood in Oakland is highlighted as an area where housing costs more than doubled during this period.\n\nThe report includes a slider that allows the user to visualize the change in monthly housing costs over time by toggling between the 2010 and 2017 data.\n\nThe m

In [27]:
response_keyword = runtime.invoke_model(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    contentType="application/json",
    accept="application/json",
    body=body_keyword
)

response_body_keyword = json.loads(response_keyword.get("body").read())

print(response_body_keyword.get("content"))

[{'type': 'text', 'text': 'The 10 keywords to describe the type of information contained in this Tableau report are:\n\n1. Affordable Housing\n2. Silicon Valley\n3. Median Monthly Costs\n4. Housing Costs\n5. Increase\n6. Geographic Distribution\n7. Census Tracts\n8. Time Comparison\n9. Oakland Neighborhood\n10. Interactive Map'}]
